# Experiment of Identification of Query Intent based on Query Log.
<img src = 'classification.jpg'>

## Fire up graphlab

In [1]:
import graphlab

In [2]:
graphlab.canvas.set_target('ipynb')

## Load data

In [3]:
data = graphlab.SFrame.read_csv('Sogou_Extraction_Features.csv')

This non-commercial license of GraphLab Create is assigned to guoxiuhe@nefu.edu.cn and will expire on April 02, 2017. For commercial licensing options, visit https://dato.com/buy/.


2016-05-04 19:12:54,257 [INFO] graphlab.cython.cy_server, 176: GraphLab Create v1.9 started. Logging: C:\Users\heguoxiu\AppData\Local\Temp\graphlab_server_1462360370.log.0


Finished parsing file E:\Python\2Graduation-Project\Sogou_Extraction_Features.csv

Parsing completed. Parsed 100 lines in 0.160009 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[str,long,str,str,str,list,long,long,str,dict,str,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file E:\Python\2Graduation-Project\Sogou_Extraction_Features.csv

Parsing completed. Parsed 10000 lines in 0.114007 secs.

## have a show of data

In [4]:
data.head()

﻿time,user_id,query,result_click,url,result_and_click,result_rank
00:00:00,2982199073774412,360安全卫士,8 3,download.it.com.cn/softweb/software/firewall/a ...,"[8, 3]",8
00:00:00,7594220010824798,哄抢救灾物资,1 1,news.21cn.com/social/daqian/2008/05/29/4777194 ...,"[1, 1]",1
00:00:00,5228056822071097,75810部队,14 5,www.greatoo.com/greatoo_cn/list.asp?link_id=27 ...,"[14, 5]",14
00:00:00,6140463203615646,绳艺,62 36,www.jd-cd.com/jd_opus/xx/200607/706.html ...,"[62, 36]",62
00:00:00,8561366108033201,汶川地震原因,3 2,www.big38.net,"[3, 2]",3
00:00:00,23908140386148713,莫衷一是的意思,1 2,www.chinabaike.com/article/81/82/110/2007/2007 ...,"[1, 2]",1
00:00:00,1797943298449139,星梦缘全集在线观��� ...,8 5,www.6wei.net/dianshiju/????\xa1\xe9|????do=index ...,"[8, 5]",8
00:00:00,717725924582846,闪字吧,1 2,www.shanziba.com,"[1, 2]",1
00:00:00,41416219018952116,霍震霆与朱玲玲照��� ...,2 6,bbs.gouzai.cn/thread-698736.html ...,"[2, 6]",2
00:00:00,9975666857142764,电脑创业,2 2,ks.cn.yahoo.com/question/1307120203719.html ...,"[2, 2]",2


## Caculate the query_word_count of the query_cut

In [5]:
data['query_word_count'] = graphlab.text_analytics.count_words(data['query_cut'])

In [6]:
data['tfidf'] = graphlab.text_analytics.tf_idf(data['query_word_count'])

## Set the train_data, validation_data, test_data

In [7]:
train_data, test_data = data.random_split(.8, seed=0)

In [8]:
train_data, validation_data = train_data.random_split(0.75, seed=0)

# Experiment Analysis

## 1. logistic_classifier

In [9]:
logistic_model = graphlab.logistic_classifier.create(train_data, target='label', features=['tfidf'], validation_set=validation_data)

Logistic regression:

--------------------------------------------------------

Number of examples          : 6010

Number of classes           : 3

Number of feature columns   : 1

...

| 5         | 8        | 1.000000  | 1.256072     | 0.858902          | 0.676063            |

| 6         | 9        | 1.000000  | 1.296074     | 0.860899          | 0.672107            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

In [10]:
logistic_model.evaluate(test_data)

{'accuracy': 0.6600609756097561,
 'auc': 0.7233940858960747,
 'confusion_matrix': Columns:
 	target_label	str
 	predicted_label	str
 	count	int
 
 Rows: 9
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |   Resource   |    Navigation   |   13  |
 | Information  |    Navigation   |   72  |
 | Information  |     Resource    |  199  |
 |  Navigation  |    Navigation   |  102  |
 |  Navigation  |   Information   |  107  |
 |  Navigation  |     Resource    |   12  |
 |   Resource   |     Resource    |  136  |
 |   Resource   |   Information   |  266  |
 | Information  |   Information   |  1061 |
 +--------------+-----------------+-------+
 [9 rows x 3 columns],
 'f1_score': 0.5413760644344618,
 'log_loss': 1.2976648242124964,
 'precision': 0.5590912683927527,
 'recall': 0.528598617152834,
 'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 	class	int
 
 Rows: 300003
 


## 2. boosted_trees_classifier

In [11]:
boost_model = graphlab.boosted_trees_classifier.create(train_data, target='label', features=['tfidf'], validation_set=validation_data)

Boosted trees classifier:

--------------------------------------------------------

Number of examples          : 6010

Number of classes           : 3

Number of feature columns   : 1

...

| 3         | 0.127008     | 0.696506          | 0.835160          | 0.672601            | 0.856580            |

| 4         | 0.171010     | 0.696672          | 0.800387          | 0.674085            | 0.826896            |

| 5         | 0.212012     | 0.698502          | 0.773593          | 0.674085            | 0.805420            |

| 6         | 0.253015     | 0.697504          | 0.755660          | 0.674580            | 0.792164            |

+-----------+--------------+-------------------+-------------------+---------------------+---------------------+

In [12]:
boost_model.evaluate(test_data)

{'accuracy': 0.6859756097560976,
 'auc': 0.7082852966264563,
 'confusion_matrix': Columns:
 	target_label	str
 	predicted_label	str
 	count	int
 
 Rows: 8
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |   Resource   |    Navigation   |   1   |
 |  Navigation  |    Navigation   |   39  |
 |  Navigation  |   Information   |  182  |
 | Information  |     Resource    |   12  |
 | Information  |    Navigation   |   17  |
 |   Resource   |     Resource    |   8   |
 |   Resource   |   Information   |  406  |
 | Information  |   Information   |  1303 |
 +--------------+-----------------+-------+
 [8 rows x 3 columns],
 'f1_score': 0.3753068663218077,
 'log_loss': 0.7529904191750523,
 'precision': 0.5910879790698322,
 'recall': 0.39132530829908574,
 'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 	class	int
 
 Rows: 300003
 
 Data:
 +-----------+-----+-----+------+--

## 3. decision_tree_classifier

In [13]:
tree_model = graphlab.decision_tree_classifier.create(train_data, target='label', features=['tfidf'], validation_set=validation_data)

Decision tree classifier:

--------------------------------------------------------

Number of examples          : 6010

Number of classes           : 3

Number of feature columns   : 1

...

+-----------+--------------+-------------------+-------------------+---------------------+---------------------+

| Iteration | Elapsed Time | Training-accuracy | Training-log_loss | Validation-accuracy | Validation-log_loss |

+-----------+--------------+-------------------+-------------------+---------------------+---------------------+

| 1         | 0.031002     | 0.693511          | 0.967202          | 0.665183            | 0.974521            |

+-----------+--------------+-------------------+-------------------+---------------------+---------------------+

In [14]:
tree_model.evaluate(test_data)

{'accuracy': 0.679369918699187,
 'auc': 0.6717311270767645,
 'confusion_matrix': Columns:
 	target_label	str
 	predicted_label	str
 	count	int
 
 Rows: 9
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |   Resource   |    Navigation   |   1   |
 |  Navigation  |    Navigation   |   22  |
 |  Navigation  |   Information   |  195  |
 | Information  |     Resource    |   17  |
 | Information  |    Navigation   |   14  |
 |  Navigation  |     Resource    |   4   |
 |   Resource   |     Resource    |   14  |
 |   Resource   |   Information   |  400  |
 | Information  |   Information   |  1301 |
 +--------------+-----------------+-------+
 [9 rows x 3 columns],
 'f1_score': 0.34627890966959,
 'log_loss': 0.9716732110494738,
 'precision': 0.5602586763979169,
 'recall': 0.37000305926599336,
 'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 	class	int
 
 Rows: 300003
 
 

## 4. nearest_neighbor_classifier

In [15]:
knn_model = graphlab.nearest_neighbor_classifier.create(train_data, target='label', features=['tfidf'])

Starting brute force nearest neighbors model training.

In [16]:
knn_model.evaluate(test_data)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1968    | 0.0166389   | 5.001ms      |

...

+--------------+---------+-------------+--------------+

| 0            | 1968    | 0.0166389   | 2ms          |

| Done         |         | 100         | 507.029ms    |

+--------------+---------+-------------+--------------+

{'accuracy': 0.6422764227642277, 'confusion_matrix': Columns:
 	target_label	str
 	predicted_label	str
 	count	int
 
 Rows: 9
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 | Information  |   Information   |  1062 |
 |   Resource   |   Information   |  290  |
 |   Resource   |    Navigation   |   50  |
 |  Navigation  |   Information   |   84  |
 | Information  |    Navigation   |  150  |
 | Information  |     Resource    |  120  |
 |  Navigation  |    Navigation   |  127  |
 |   Resource   |     Resource    |   75  |
 |  Navigation  |     Resource    |   10  |
 +--------------+-----------------+-------+
 [9 rows x 3 columns]}

## 5. random_forest_classifier

In [17]:
rf_model = graphlab.random_forest_classifier.create(train_data, target='label', features=['tfidf'], validation_set=validation_data)

Random forest classifier:

--------------------------------------------------------

Number of examples          : 6010

Number of classes           : 3

Number of feature columns   : 1

...

| 3         | 0.110007     | 0.689517          | 0.787223          | 0.674580            | 0.813828            |

| 4         | 0.143009     | 0.691847          | 0.787528          | 0.676558            | 0.814237            |

| 5         | 0.187011     | 0.688186          | 0.786888          | 0.674580            | 0.813492            |

| 6         | 0.222013     | 0.691847          | 0.787143          | 0.677052            | 0.814426            |

+-----------+--------------+-------------------+-------------------+---------------------+---------------------+

In [18]:
rf_model.evaluate(test_data)

{'accuracy': 0.6875, 'auc': 0.6923104953105416, 'confusion_matrix': Columns:
 	target_label	str
 	predicted_label	str
 	count	int
 
 Rows: 7
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |  Navigation  |    Navigation   |   33  |
 |  Navigation  |   Information   |  188  |
 |   Resource   |     Resource    |   8   |
 |   Resource   |   Information   |  407  |
 | Information  |   Information   |  1312 |
 | Information  |    Navigation   |   17  |
 | Information  |     Resource    |   3   |
 +--------------+-----------------+-------+
 [7 rows x 3 columns], 'f1_score': 0.36374256771628155, 'log_loss': 0.8019730761225622, 'precision': 0.6917547790437145, 'recall': 0.38452778679568184, 'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 	class	int
 
 Rows: 300003
 
 Data:
 +-----------+-----+-----+------+-----+-------+
 | threshold | fpr | tpr |  p   |  n  | class |
 